In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 1000

In [3]:
df = pd.read_csv('D:/cap/capstone2/data/interim/preprocessing.csv')
to_drop = ['Unnamed: 0']
df.drop(to_drop, axis=1,inplace=True)

In [4]:
df.head()

,event_cause,crew_category,crew_age,med_certf,med_crtf_vldty,light_cond,wx_cond_basic,type_fly,pilot_privileges,CERT_FORN,CERT_MIL,AP_MEL,AP_MES,AP_SEL,AP_SES,AP_NONE,INSTRUCT_AME,INSTRUCT_ASE,INSTRUCT_IAIR,INSTRUCT_SPRT,INSTRUM_APLN,ACTU-INST,ACTU-IRCV,ACTU-L24H,ACTU-L30D,ACTU-L90D,ACTU-PIC,ACTU-TOTL,ALL-INSTRUM,ALL-IRCV,ALL-L24H,ALL-L30D,ALL-L90D,ALL-PIC,ALL-TOTL,MAKE-INSTRUCT,MAKE-IRCV,MAKE-L24H,MAKE-L30D,MAKE-L90D,MAKE-PIC,MAKE-TOTL,MENG-INSTRUCT,MENG-IRCV,MENG-L24H,MENG-L30D,MENG-L90D,MENG-PIC,MENG-TOTL,NGHT-INSTRUCT,NGHT-IRCV,NGHT-L24H,NGHT-L30D,NGHT-L90D,NGHT-PIC,NGHT-TOTL,SENG-INSTRUCT,SENG-IRCV,SENG-L24H,SENG-L30D,SENG-L90D,SENG-PIC,SENG-TOTL,SIMU-TOTL
0,Personnel-Decision making/judgment,PILOT,46,CL3,UNK,DAYL,IMC,PERS,PRIVATE,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,15.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0
1,Enviro-Ceiling/visibility/precip,PILOT,46,CL3,UNK,DAYL,IMC,PERS,PRIVATE,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,15.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0
2,Enviro-Ceiling/visibility/precip,PILOT,46,CL3,UNK,DAYL,IMC,PERS,PRIVATE,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,15.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0
3,Aircraft oper/perf/capability-Altitude,PILOT,46,CL3,UNK,DAYL,IMC,PERS,PRIVATE,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,15.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0
4,Aircraft oper/perf/capability-(general),PILOT,46,CL3,UNK,DAYL,IMC,PERS,PRIVATE,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,15.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,275.0,0.0


In [5]:
df.shape

(30774, 64)

In [6]:
df.drop_duplicates( keep='first', inplace=True, ignore_index=True)

In [7]:
df.shape

(27935, 64)

In [8]:
events_count = df.event_cause.value_counts()
events_count

Personnel-Aircraft control                                                     4310
Aircraft oper/perf/capability-Directional control                              1843
Enviro-External Obstacle                                                       1616
Personnel-Incorrect Action                                                     1555
Enviro-Wind                                                                    1431
Personnel-Decision making/judgment                                             1288
Aircraft oper/perf/capability-Airspeed                                          812
Enviro-Terrain                                                                  777
Fluids/misc hardware-Fuel                                                       723
Aircraft oper/perf/capability-(general)                                         722
Aircraft oper/perf/capability-Landing flare                                     672
Personnel-Use of equip/system                                               

In [9]:
# Keep events that have at least 40 data points
high_freq = df[df['event_cause'].map(df['event_cause'].value_counts()) >= 1400]

In [10]:
events_count = high_freq.event_cause.value_counts()
events_count

Personnel-Aircraft control                           4310
Aircraft oper/perf/capability-Directional control    1843
Enviro-External Obstacle                             1616
Personnel-Incorrect Action                           1555
Enviro-Wind                                          1431
Name: event_cause, dtype: int64

#### Normalize Target (event_cause) labels

In [11]:
le = LabelEncoder()
high_freq['event_coded'] = le.fit_transform(high_freq['event_cause'])
high_freq.head(3)



<ipython-input-11-27b30e639828>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_freq['event_coded'] = le.fit_transform(high_freq['event_cause'])


,event_cause,crew_category,crew_age,med_certf,med_crtf_vldty,light_cond,wx_cond_basic,type_fly,pilot_privileges,CERT_FORN,CERT_MIL,AP_MEL,AP_MES,AP_SEL,AP_SES,AP_NONE,INSTRUCT_AME,INSTRUCT_ASE,INSTRUCT_IAIR,INSTRUCT_SPRT,INSTRUM_APLN,ACTU-INST,ACTU-IRCV,ACTU-L24H,ACTU-L30D,ACTU-L90D,ACTU-PIC,ACTU-TOTL,ALL-INSTRUM,ALL-IRCV,ALL-L24H,ALL-L30D,ALL-L90D,ALL-PIC,ALL-TOTL,MAKE-INSTRUCT,MAKE-IRCV,MAKE-L24H,MAKE-L30D,MAKE-L90D,MAKE-PIC,MAKE-TOTL,MENG-INSTRUCT,MENG-IRCV,MENG-L24H,MENG-L30D,MENG-L90D,MENG-PIC,MENG-TOTL,NGHT-INSTRUCT,NGHT-IRCV,NGHT-L24H,NGHT-L30D,NGHT-L90D,NGHT-PIC,NGHT-TOTL,SENG-INSTRUCT,SENG-IRCV,SENG-L24H,SENG-L30D,SENG-L90D,SENG-PIC,SENG-TOTL,SIMU-TOTL,event_coded
4,Enviro-Wind,PILOT,54,CL3,VALID,DAYL,VMC,PERS,COMMERCIAL,0,0,0,0,1,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,42.0,42.0,10.0,0.0,2.0,4.0,16.0,870.0,1000.0,0.0,0.0,2.0,4.0,16.0,260.0,260.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,53.0,0.0,0.0,2.0,4.0,16.0,870.0,1000.0,85.0,2
5,Aircraft oper/perf/capability-Directional control,PILOT,54,CL3,VALID,DAYL,VMC,PERS,COMMERCIAL,0,0,0,0,1,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,42.0,42.0,10.0,0.0,2.0,4.0,16.0,870.0,1000.0,0.0,0.0,2.0,4.0,16.0,260.0,260.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,53.0,0.0,0.0,2.0,4.0,16.0,870.0,1000.0,85.0,0
9,Personnel-Aircraft control,PILOT,31,CL3,VALID,DAYL,VMC,PERS,PRIVATE,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,107.0,172.0,0.0,0.0,0.0,0.0,3.0,89.0,153.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,172.0,0.0,3


In [12]:
event_code = high_freq[['event_cause', 'event_coded']]
event_code.to_csv('D:/cap/capstone2/data/interim/target_codes.csv')

In [13]:
df = high_freq.drop(columns=['event_cause'], axis=1)

In [14]:
popped = df.pop('event_coded')
df.insert(0, 'event_coded',popped)


In [15]:
df.head()

,event_coded,crew_category,crew_age,med_certf,med_crtf_vldty,light_cond,wx_cond_basic,type_fly,pilot_privileges,CERT_FORN,CERT_MIL,AP_MEL,AP_MES,AP_SEL,AP_SES,AP_NONE,INSTRUCT_AME,INSTRUCT_ASE,INSTRUCT_IAIR,INSTRUCT_SPRT,INSTRUM_APLN,ACTU-INST,ACTU-IRCV,ACTU-L24H,ACTU-L30D,ACTU-L90D,ACTU-PIC,ACTU-TOTL,ALL-INSTRUM,ALL-IRCV,ALL-L24H,ALL-L30D,ALL-L90D,ALL-PIC,ALL-TOTL,MAKE-INSTRUCT,MAKE-IRCV,MAKE-L24H,MAKE-L30D,MAKE-L90D,MAKE-PIC,MAKE-TOTL,MENG-INSTRUCT,MENG-IRCV,MENG-L24H,MENG-L30D,MENG-L90D,MENG-PIC,MENG-TOTL,NGHT-INSTRUCT,NGHT-IRCV,NGHT-L24H,NGHT-L30D,NGHT-L90D,NGHT-PIC,NGHT-TOTL,SENG-INSTRUCT,SENG-IRCV,SENG-L24H,SENG-L30D,SENG-L90D,SENG-PIC,SENG-TOTL,SIMU-TOTL
4,2,PILOT,54,CL3,VALID,DAYL,VMC,PERS,COMMERCIAL,0,0,0,0,1,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,42.0,42.0,10.0,0.0,2.0,4.0,16.0,870.0,1000.0,0.0,0.0,2.0,4.0,16.0,260.0,260.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,53.0,0.0,0.0,2.0,4.0,16.0,870.0,1000.0,85.0
5,0,PILOT,54,CL3,VALID,DAYL,VMC,PERS,COMMERCIAL,0,0,0,0,1,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,42.0,42.0,10.0,0.0,2.0,4.0,16.0,870.0,1000.0,0.0,0.0,2.0,4.0,16.0,260.0,260.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,53.0,0.0,0.0,2.0,4.0,16.0,870.0,1000.0,85.0
9,3,PILOT,31,CL3,VALID,DAYL,VMC,PERS,PRIVATE,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,107.0,172.0,0.0,0.0,0.0,0.0,3.0,89.0,153.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,172.0,0.0
12,4,PILOT,65,CL3,VALID,DAYL,IMC,PERS,PRIVATE,0,0,0,0,1,0,0,0,0,0,0,1,0.0,30.0,0.0,0.0,0.0,0.0,41.0,3.0,608.0,0.0,5.0,26.0,15.0,1049.0,0.0,51.0,0.0,5.0,26.0,0.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,598.0,0.0,5.0,26.0,0.0,1049.0,63.0
14,4,PILOT,73,CL3,VALID,DAYL,VMC,PERS,COMMERCIAL,0,0,0,0,1,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,1.0,9.0,2658.0,4190.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# OneHot Encode Categorical features

df = pd.get_dummies(df, columns=['crew_category'], prefix='crew', drop_first=False)
df = pd.get_dummies(df, columns=['med_certf'], prefix='med_certf', drop_first=False)
df = pd.get_dummies(df, columns=['med_crtf_vldty'], prefix='med_crtf_vldty', drop_first=False)
df = pd.get_dummies(df, columns=['light_cond'], prefix='light_cond', drop_first=False)
df = pd.get_dummies(df, columns=['wx_cond_basic'], prefix='wx_cond_basic', drop_first=False)
df = pd.get_dummies(df, columns=['type_fly'], prefix='type_fly', drop_first=False)
df = pd.get_dummies(df, columns=['pilot_privileges'], prefix='pilot_privileges', drop_first=False)


In [17]:
df.head(2)

,event_coded,crew_age,CERT_FORN,CERT_MIL,AP_MEL,AP_MES,AP_SEL,AP_SES,AP_NONE,INSTRUCT_AME,INSTRUCT_ASE,INSTRUCT_IAIR,INSTRUCT_SPRT,INSTRUM_APLN,ACTU-INST,ACTU-IRCV,ACTU-L24H,ACTU-L30D,ACTU-L90D,ACTU-PIC,ACTU-TOTL,ALL-INSTRUM,ALL-IRCV,ALL-L24H,ALL-L30D,ALL-L90D,ALL-PIC,ALL-TOTL,MAKE-INSTRUCT,MAKE-IRCV,MAKE-L24H,MAKE-L30D,MAKE-L90D,MAKE-PIC,MAKE-TOTL,MENG-INSTRUCT,MENG-IRCV,MENG-L24H,MENG-L30D,MENG-L90D,MENG-PIC,MENG-TOTL,NGHT-INSTRUCT,NGHT-IRCV,NGHT-L24H,NGHT-L30D,NGHT-L90D,NGHT-PIC,NGHT-TOTL,SENG-INSTRUCT,SENG-IRCV,SENG-L24H,SENG-L30D,SENG-L90D,SENG-PIC,SENG-TOTL,SIMU-TOTL,crew_PILOT,crew_STUDENT,med_certf_CL1,med_certf_CL2,med_certf_CL3,med_certf_NONE,med_certf_UNK,med_crtf_vldty_INVALID,med_crtf_vldty_UNK,med_crtf_vldty_VALID,light_cond_DAWN,light_cond_DAYL,light_cond_DUSK,light_cond_NBRT,light_cond_NDRK,light_cond_NITE,light_cond_NR,wx_cond_basic_IMC,wx_cond_basic_UNK,wx_cond_basic_VMC,type_fly_AERIALOBS,type_fly_AIRDROP,type_fly_ASHOW,type_fly_BANNTOW,type_fly_BUSS,type_fly_CROPDUST,type_fly_EXTERNALLOAD,type_fly_FERRY,type_fly_FLTTEST,type_fly_GLDTTOW,type_fly_INSTRUC,type_fly_OTHRWORK,type_fly_PERS,type_fly_PUBLIC,type_fly_SKYD,type_fly_UNK,pilot_privileges_ATP,pilot_privileges_COMMERCIAL,pilot_privileges_NONE,pilot_privileges_PRIVATE,pilot_privileges_SPRT/REC,pilot_privileges_STUDENT
4,2,54,0,0,0,0,1,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,42.0,42.0,10.0,0.0,2.0,4.0,16.0,870.0,1000.0,0.0,0.0,2.0,4.0,16.0,260.0,260.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,53.0,0.0,0.0,2.0,4.0,16.0,870.0,1000.0,85.0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
5,0,54,0,0,0,0,1,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,42.0,42.0,10.0,0.0,2.0,4.0,16.0,870.0,1000.0,0.0,0.0,2.0,4.0,16.0,260.0,260.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,53.0,0.0,0.0,2.0,4.0,16.0,870.0,1000.0,85.0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0


In [18]:
scaler = MinMaxScaler()
scaler.fit(df[['crew_age']])
df['normed_age'] = scaler.transform(df[['crew_age']])

In [19]:
df.head()

,event_coded,crew_age,CERT_FORN,CERT_MIL,AP_MEL,AP_MES,AP_SEL,AP_SES,AP_NONE,INSTRUCT_AME,INSTRUCT_ASE,INSTRUCT_IAIR,INSTRUCT_SPRT,INSTRUM_APLN,ACTU-INST,ACTU-IRCV,ACTU-L24H,ACTU-L30D,ACTU-L90D,ACTU-PIC,ACTU-TOTL,ALL-INSTRUM,ALL-IRCV,ALL-L24H,ALL-L30D,ALL-L90D,ALL-PIC,ALL-TOTL,MAKE-INSTRUCT,MAKE-IRCV,MAKE-L24H,MAKE-L30D,MAKE-L90D,MAKE-PIC,MAKE-TOTL,MENG-INSTRUCT,MENG-IRCV,MENG-L24H,MENG-L30D,MENG-L90D,MENG-PIC,MENG-TOTL,NGHT-INSTRUCT,NGHT-IRCV,NGHT-L24H,NGHT-L30D,NGHT-L90D,NGHT-PIC,NGHT-TOTL,SENG-INSTRUCT,SENG-IRCV,SENG-L24H,SENG-L30D,SENG-L90D,SENG-PIC,SENG-TOTL,SIMU-TOTL,crew_PILOT,crew_STUDENT,med_certf_CL1,med_certf_CL2,med_certf_CL3,med_certf_NONE,med_certf_UNK,med_crtf_vldty_INVALID,med_crtf_vldty_UNK,med_crtf_vldty_VALID,light_cond_DAWN,light_cond_DAYL,light_cond_DUSK,light_cond_NBRT,light_cond_NDRK,light_cond_NITE,light_cond_NR,wx_cond_basic_IMC,wx_cond_basic_UNK,wx_cond_basic_VMC,type_fly_AERIALOBS,type_fly_AIRDROP,type_fly_ASHOW,type_fly_BANNTOW,type_fly_BUSS,type_fly_CROPDUST,type_fly_EXTERNALLOAD,type_fly_FERRY,type_fly_FLTTEST,type_fly_GLDTTOW,type_fly_INSTRUC,type_fly_OTHRWORK,type_fly_PERS,type_fly_PUBLIC,type_fly_SKYD,type_fly_UNK,pilot_privileges_ATP,pilot_privileges_COMMERCIAL,pilot_privileges_NONE,pilot_privileges_PRIVATE,pilot_privileges_SPRT/REC,pilot_privileges_STUDENT,normed_age
4,2,54,0,0,0,0,1,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,42.0,42.0,10.0,0.0,2.0,4.0,16.0,870.0,1000.0,0.0,0.0,2.0,4.0,16.0,260.0,260.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,53.0,0.0,0.0,2.0,4.0,16.0,870.0,1000.0,85.0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0.5000
5,0,54,0,0,0,0,1,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,42.0,42.0,10.0,0.0,2.0,4.0,16.0,870.0,1000.0,0.0,0.0,2.0,4.0,16.0,260.0,260.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,53.0,0.0,0.0,2.0,4.0,16.0,870.0,1000.0,85.0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0.5000
9,3,31,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,107.0,172.0,0.0,0.0,0.0,0.0,3.0,89.0,153.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,172.0,0.0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0.2125
12,4,65,0,0,0,0,1,0,0,0,0,0,0,1,0.0,30.0,0.0,0.0,0.0,0.0,41.0,3.0,608.0,0.0,5.0,26.0,15.0,1049.0,0.0,51.0,0.0,5.0,26.0,0.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,598.0,0.0,5.0,26.0,0.0,1049.0,63.0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0.6375
14,4,73,0,0,0,0,1,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,1.0,9.0,2658.0,4190.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0.7375


In [20]:
nominals = ['ACTU-INST', 'ACTU-IRCV', 'ACTU-L24H', 'ACTU-L30D', 'ACTU-L90D', 'ACTU-PIC', 'ACTU-TOTL', 'ALL-INSTRUM', 
            'ALL-IRCV', 'ALL-L24H', 'ALL-L30D', 'ALL-L90D', 'ALL-PIC', 'ALL-TOTL', 'MAKE-INSTRUCT', 'MAKE-IRCV',
            'MAKE-L24H', 'MAKE-L30D', 'MAKE-L90D', 'MAKE-PIC', 'MAKE-TOTL', 'MENG-INSTRUCT', 'MENG-IRCV', 'MENG-L24H',
            'MENG-L30D', 'MENG-L90D', 'MENG-PIC', 'MENG-TOTL', 'NGHT-INSTRUCT', 'NGHT-IRCV', 'NGHT-L24H',
            'NGHT-L30D', 'NGHT-L90D', 'NGHT-PIC', 'NGHT-TOTL', 'SENG-INSTRUCT', 'SENG-IRCV', 'SENG-L24H', 'SENG-L30D', 
            'SENG-L90D', 'SENG-PIC', 'SENG-TOTL', 'SIMU-TOTL',]

In [21]:
scaler = StandardScaler()

for nominal in nominals:
    scaler.fit(df[[nominal]])
    df[nominal] = scaler.transform(df[[nominal]])    

In [22]:
df.head()

,event_coded,crew_age,CERT_FORN,CERT_MIL,AP_MEL,AP_MES,AP_SEL,AP_SES,AP_NONE,INSTRUCT_AME,INSTRUCT_ASE,INSTRUCT_IAIR,INSTRUCT_SPRT,INSTRUM_APLN,ACTU-INST,ACTU-IRCV,ACTU-L24H,ACTU-L30D,ACTU-L90D,ACTU-PIC,ACTU-TOTL,ALL-INSTRUM,ALL-IRCV,ALL-L24H,ALL-L30D,ALL-L90D,ALL-PIC,ALL-TOTL,MAKE-INSTRUCT,MAKE-IRCV,MAKE-L24H,MAKE-L30D,MAKE-L90D,MAKE-PIC,MAKE-TOTL,MENG-INSTRUCT,MENG-IRCV,MENG-L24H,MENG-L30D,MENG-L90D,MENG-PIC,MENG-TOTL,NGHT-INSTRUCT,NGHT-IRCV,NGHT-L24H,NGHT-L30D,NGHT-L90D,NGHT-PIC,NGHT-TOTL,SENG-INSTRUCT,SENG-IRCV,SENG-L24H,SENG-L30D,SENG-L90D,SENG-PIC,SENG-TOTL,SIMU-TOTL,crew_PILOT,crew_STUDENT,med_certf_CL1,med_certf_CL2,med_certf_CL3,med_certf_NONE,med_certf_UNK,med_crtf_vldty_INVALID,med_crtf_vldty_UNK,med_crtf_vldty_VALID,light_cond_DAWN,light_cond_DAYL,light_cond_DUSK,light_cond_NBRT,light_cond_NDRK,light_cond_NITE,light_cond_NR,wx_cond_basic_IMC,wx_cond_basic_UNK,wx_cond_basic_VMC,type_fly_AERIALOBS,type_fly_AIRDROP,type_fly_ASHOW,type_fly_BANNTOW,type_fly_BUSS,type_fly_CROPDUST,type_fly_EXTERNALLOAD,type_fly_FERRY,type_fly_FLTTEST,type_fly_GLDTTOW,type_fly_INSTRUC,type_fly_OTHRWORK,type_fly_PERS,type_fly_PUBLIC,type_fly_SKYD,type_fly_UNK,pilot_privileges_ATP,pilot_privileges_COMMERCIAL,pilot_privileges_NONE,pilot_privileges_PRIVATE,pilot_privileges_SPRT/REC,pilot_privileges_STUDENT,normed_age
4,2,54,0,0,0,0,1,0,0,0,0,0,0,1,-0.040955,-0.017131,-0.080016,-0.124221,-0.109637,0.152596,0.032552,-0.021728,-0.062861,0.022006,-0.136661,-0.116987,0.122867,-0.152750,-0.054806,-0.057549,0.604889,-0.205759,0.015518,0.198064,-0.041983,-0.072699,-0.016469,-0.095458,-0.131082,-0.130166,-0.033299,-0.216076,-0.078108,-0.039866,-0.088823,-0.161534,-0.157168,0.128263,-0.030947,-0.128212,-0.043184,0.778204,-0.141473,0.072593,0.438045,0.266305,0.593982,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0.5000
5,0,54,0,0,0,0,1,0,0,0,0,0,0,1,-0.040955,-0.017131,-0.080016,-0.124221,-0.109637,0.152596,0.032552,-0.021728,-0.062861,0.022006,-0.136661,-0.116987,0.122867,-0.152750,-0.054806,-0.057549,0.604889,-0.205759,0.015518,0.198064,-0.041983,-0.072699,-0.016469,-0.095458,-0.131082,-0.130166,-0.033299,-0.216076,-0.078108,-0.039866,-0.088823,-0.161534,-0.157168,0.128263,-0.030947,-0.128212,-0.043184,0.778204,-0.141473,0.072593,0.438045,0.266305,0.593982,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0.5000
9,3,31,0,0,0,0,1,0,0,0,0,0,0,0,-0.040955,-0.017131,-0.080016,-0.124221,-0.109637,-0.158936,-0.212100,-0.022818,-0.062861,-0.041622,-0.254503,-0.395593,-0.466653,-0.689463,-0.054806,-0.057549,-0.491139,-0.591405,-0.115289,-0.175874,-0.228521,-0.072699,-0.016469,-0.095458,-0.131082,-0.130166,-0.033299,-0.216076,-0.078108,-0.039866,-0.088823,-0.161534,-0.157168,-0.220079,-0.310152,-0.128212,-0.043184,-0.442045,-0.480703,-0.367573,-0.440641,-0.425580,-0.240063,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0.2125
12,4,65,0,0,0,0,1,0,0,0,0,0,0,1,-0.040955,4.776734,-0.080016,-0.124221,-0.109637,-0.158936,0.026727,-0.022818,28.398130,-0.041622,-0.107201,0.097325,-0.537736,-0.120988,-0.054806,13.543212,-0.491139,-0.109348,0.116138,-0.370497,-0.394139,-0.072699,-0.016469,-0.095458,-0.131082,-0.130166,-0.033299,-0.216076,-0.078108,-0.039866,-0.088823,-0.161534,-0.157168,-0.220079,-0.310152,-0.128212,39.804538,-0.442045,-0.056665,0.411183,-0.440641,0.307250,0.378112,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0.6375
14,4,73,0,0,0,0,1,0,0,0,0,0,0,1,-0.040955,-0.017131,-0.080016,-0.124221,-0.109637,-0.158936,-0.212100,-0.021728,-0.062861,-0.041622,-0.225043,-0.267006,1.504338,1.915020,-0.054806,-0.057549,-0.491139,-0.591405,-0.145475,-0.370497,-0.495253,-0.072699,-0.016469,-0.095458,-0.131082,-0.130166,-0.033299,-0.216076,-0.078108,-0.039866,-0.088823,-0.161534,-0.157168,-0.220079,-0.298984,-0.128212,-0.043184,-0.442045,-0.480703,-0.469150,-0.440641,-0.569305,-0.240063,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0

In [23]:
df.drop(columns='crew_age', inplace=True)

In [24]:
df.head()

,event_coded,CERT_FORN,CERT_MIL,AP_MEL,AP_MES,AP_SEL,AP_SES,AP_NONE,INSTRUCT_AME,INSTRUCT_ASE,INSTRUCT_IAIR,INSTRUCT_SPRT,INSTRUM_APLN,ACTU-INST,ACTU-IRCV,ACTU-L24H,ACTU-L30D,ACTU-L90D,ACTU-PIC,ACTU-TOTL,ALL-INSTRUM,ALL-IRCV,ALL-L24H,ALL-L30D,ALL-L90D,ALL-PIC,ALL-TOTL,MAKE-INSTRUCT,MAKE-IRCV,MAKE-L24H,MAKE-L30D,MAKE-L90D,MAKE-PIC,MAKE-TOTL,MENG-INSTRUCT,MENG-IRCV,MENG-L24H,MENG-L30D,MENG-L90D,MENG-PIC,MENG-TOTL,NGHT-INSTRUCT,NGHT-IRCV,NGHT-L24H,NGHT-L30D,NGHT-L90D,NGHT-PIC,NGHT-TOTL,SENG-INSTRUCT,SENG-IRCV,SENG-L24H,SENG-L30D,SENG-L90D,SENG-PIC,SENG-TOTL,SIMU-TOTL,crew_PILOT,crew_STUDENT,med_certf_CL1,med_certf_CL2,med_certf_CL3,med_certf_NONE,med_certf_UNK,med_crtf_vldty_INVALID,med_crtf_vldty_UNK,med_crtf_vldty_VALID,light_cond_DAWN,light_cond_DAYL,light_cond_DUSK,light_cond_NBRT,light_cond_NDRK,light_cond_NITE,light_cond_NR,wx_cond_basic_IMC,wx_cond_basic_UNK,wx_cond_basic_VMC,type_fly_AERIALOBS,type_fly_AIRDROP,type_fly_ASHOW,type_fly_BANNTOW,type_fly_BUSS,type_fly_CROPDUST,type_fly_EXTERNALLOAD,type_fly_FERRY,type_fly_FLTTEST,type_fly_GLDTTOW,type_fly_INSTRUC,type_fly_OTHRWORK,type_fly_PERS,type_fly_PUBLIC,type_fly_SKYD,type_fly_UNK,pilot_privileges_ATP,pilot_privileges_COMMERCIAL,pilot_privileges_NONE,pilot_privileges_PRIVATE,pilot_privileges_SPRT/REC,pilot_privileges_STUDENT,normed_age
4,2,0,0,0,0,1,0,0,0,0,0,0,1,-0.040955,-0.017131,-0.080016,-0.124221,-0.109637,0.152596,0.032552,-0.021728,-0.062861,0.022006,-0.136661,-0.116987,0.122867,-0.152750,-0.054806,-0.057549,0.604889,-0.205759,0.015518,0.198064,-0.041983,-0.072699,-0.016469,-0.095458,-0.131082,-0.130166,-0.033299,-0.216076,-0.078108,-0.039866,-0.088823,-0.161534,-0.157168,0.128263,-0.030947,-0.128212,-0.043184,0.778204,-0.141473,0.072593,0.438045,0.266305,0.593982,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0.5000
5,0,0,0,0,0,1,0,0,0,0,0,0,1,-0.040955,-0.017131,-0.080016,-0.124221,-0.109637,0.152596,0.032552,-0.021728,-0.062861,0.022006,-0.136661,-0.116987,0.122867,-0.152750,-0.054806,-0.057549,0.604889,-0.205759,0.015518,0.198064,-0.041983,-0.072699,-0.016469,-0.095458,-0.131082,-0.130166,-0.033299,-0.216076,-0.078108,-0.039866,-0.088823,-0.161534,-0.157168,0.128263,-0.030947,-0.128212,-0.043184,0.778204,-0.141473,0.072593,0.438045,0.266305,0.593982,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0.5000
9,3,0,0,0,0,1,0,0,0,0,0,0,0,-0.040955,-0.017131,-0.080016,-0.124221,-0.109637,-0.158936,-0.212100,-0.022818,-0.062861,-0.041622,-0.254503,-0.395593,-0.466653,-0.689463,-0.054806,-0.057549,-0.491139,-0.591405,-0.115289,-0.175874,-0.228521,-0.072699,-0.016469,-0.095458,-0.131082,-0.130166,-0.033299,-0.216076,-0.078108,-0.039866,-0.088823,-0.161534,-0.157168,-0.220079,-0.310152,-0.128212,-0.043184,-0.442045,-0.480703,-0.367573,-0.440641,-0.425580,-0.240063,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0.2125
12,4,0,0,0,0,1,0,0,0,0,0,0,1,-0.040955,4.776734,-0.080016,-0.124221,-0.109637,-0.158936,0.026727,-0.022818,28.398130,-0.041622,-0.107201,0.097325,-0.537736,-0.120988,-0.054806,13.543212,-0.491139,-0.109348,0.116138,-0.370497,-0.394139,-0.072699,-0.016469,-0.095458,-0.131082,-0.130166,-0.033299,-0.216076,-0.078108,-0.039866,-0.088823,-0.161534,-0.157168,-0.220079,-0.310152,-0.128212,39.804538,-0.442045,-0.056665,0.411183,-0.440641,0.307250,0.378112,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0.6375
14,4,0,0,0,0,1,0,0,0,0,0,0,1,-0.040955,-0.017131,-0.080016,-0.124221,-0.109637,-0.158936,-0.212100,-0.021728,-0.062861,-0.041622,-0.225043,-0.267006,1.504338,1.915020,-0.054806,-0.057549,-0.491139,-0.591405,-0.145475,-0.370497,-0.495253,-0.072699,-0.016469,-0.095458,-0.131082,-0.130166,-0.033299,-0.216076,-0.078108,-0.039866,-0.088823,-0.161534,-0.157168,-0.220079,-0.298984,-0.128212,-0.043184,-0.442045,-0.480703,-0.469150,-0.440641,-0.569305,-0.240063,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10755 entries, 4 to 27933
Data columns (total 99 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   event_coded                  10755 non-null  int32  
 1   CERT_FORN                    10755 non-null  int64  
 2   CERT_MIL                     10755 non-null  int64  
 3   AP_MEL                       10755 non-null  int64  
 4   AP_MES                       10755 non-null  int64  
 5   AP_SEL                       10755 non-null  int64  
 6   AP_SES                       10755 non-null  int64  
 7   AP_NONE                      10755 non-null  int64  
 8   INSTRUCT_AME                 10755 non-null  int64  
 9   INSTRUCT_ASE                 10755 non-null  int64  
 10  INSTRUCT_IAIR                10755 non-null  int64  
 11  INSTRUCT_SPRT                10755 non-null  int64  
 12  INSTRUM_APLN                 10755 non-null  int64  
 13  ACTU-INST       

In [26]:
df.to_csv('D:/cap/capstone2/data/processed/processed.csv', index=False)